In [1]:
from googleapiclient.discovery import build
from selenium import webdriver
from selenium.webdriver.common.by import By
import os, time
from selenium.webdriver.chrome.service import Service
from datetime import datetime as dt
import googleapiclient.errors

In [2]:
DEVELOPER_KEY = YOUR_API_KEY
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
  
# creating youtube resource object
# for interacting with API
youtube = build(YOUTUBE_API_SERVICE_NAME, 
                     YOUTUBE_API_VERSION,
            developerKey = DEVELOPER_KEY)

### **This function will return the channel ID by passing the name of youtuber**

In [ ]:
def get_channel_ID(youtube, channel_name):
    
    try:
        search_response = youtube.search().list(
        q=channel_name,
        type="channel",
        part="id"
        ).execute()
    
        channel_ID = search_response['items'][0]['id']['channelId']
        return channel_ID
    
    except googleapiclient.errors.HttpError as e:
        status_code = e.resp.status
        error_message = e.content.decode("utf-8")
        print(f"HTTP status code: {status_code}")
        print(f"Error message: {error_message}")

### **This function will return the channel code by passing the name of youtuber**

In [ ]:
def get_playlistID(youtube, channel_name):

    try:
        chrome_service = Service(executable_path='chromedriver.exe')
        wd = webdriver.Chrome(service=chrome_service)
        sleep_between_interactions = 1

        url = "https://www.youtube.com/results?search_query={}".format(channel_name.replace(" ", ""))
        print(url)
        wd.get(url)
        time.sleep(5)

        search_response = youtube.search().list(
        q=channel_name,
        type="channel",
        part="id"
        ).execute()
        
        # video_url = wd.find_element(By.ID, "main-link").get_attribute("href")
        # print(video_url)   
        # wd.get(video_url)
        # time.sleep(5)
        # channel_id = wd.find_element(By.XPATH, '//*[@id="first-link"]/span/a').get_attribute('href').split("channel/")[-1]
        # wd.quit()
        # return channel_id
        
        # video_url = wd.find_element(By.ID, "main-link").get_attribute("href")
        # username = video_url.split("@")[1]
        # wd.quit()
        # return username



    except googleapiclient.errors.HttpError as e:
        status_code = e.resp.status
        error_message = e.content.decode("utf-8")
        print(f"HTTP status code: {status_code}")
        print(f"Error message: {error_message}")

In [ ]:
playlist_id = get_playlistID(youtube, "technical guruji")
playlist_id

In [ ]:
from googleapiclient.errors import HttpError
try:
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id = playlist_id
    )
    response = request.execute()

    print(response)

except HttpError as e:
    print(f"An HTTP error occurred: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

### **this function will return the channel info using the channel code from above function**

In [ ]:
def get_channel_info(youtube, username):

    try:
        channel_info = {}
        channel_response = youtube.channels().list(
            part="snippet,contentDetails,statistics",
            forUsername = username
        ).execute()
        
        channel_info = dict(
        channel_id = channel_response.get("items", [])[0]['id'],
        channel_name = channel_response.get("items", [])[0]['snippet']['title'],
        playlist_ID = channel_response.get("items", [])[0]['contentDetails']['relatedPlaylists']['uploads']
        # subscriber_count = channel_response.get("items", [])[0]['statistics']['subscriberCount'],
        # video_count = channel_response.get("items", [])[0]['statistics']['videoCount']
        )

        return channel_info
    
    except googleapiclient.errors.HttpError as e:
        status_code = e.resp.status
        error_message = e.content.decode("utf-8")
        print(f"HTTP status code: {status_code}")
        print(f"Error message: {error_message}")

In [ ]:
channel_info = get_channel_info(youtube, username=username1)
channel_info

### **this function will retrive all the video IDs present in the channel and return the dict with first 100 video IDs**

In [ ]:
def get_all_video_ID(youtube, playlistid):
    
    try:
        video_details = []
        video_ids = []
        request_video_ID = youtube.playlistItems().list(
            part="contentDetails,snippet",
            maxResults=50,
            playlistId = playlistid,
        )

        response = request_video_ID.execute()
        video_details.extend(response["items"])

        while "nextPageToken" in response:
                nextPageToken = response["nextPageToken"]
                request = youtube.playlistItems().list(
                    part="contentDetails,snippet",
                    maxResults=50,
                    playlistId = playlistid,
                    pageToken=nextPageToken,
                )
                response = request.execute()
                video_details.extend(response["items"])
        
        for i in range(len(video_details)):
            video_ids.append(video_details[i]['contentDetails']['videoId'])

        return dict(channel_id = playlistid,
                    video_ids = video_ids[:100])
        
    except googleapiclient.errors.HttpError as e:
        status_code = e.resp.status
        error_message = e.content.decode("utf-8")
        print(f"HTTP status code: {status_code}")
        print(f"Error message: {error_message}")

In [ ]:
channel_video_dict = get_all_video_ID(youtube, channel_info['playlist_ID'])

In [ ]:
channel_video_dict

In [ ]:
def get_video_details(video_dict):
        try:
                video_info = []
                request = youtube.videos().list(
                        part = 'snippet,statistics,contentDetails',
                        id = ",".join(video_dict['video_ids'][:50])
                )
                response = request.execute()
                video_info.extend(response["items"])


                request1 = youtube.videos().list(
                        part = 'snippet,statistics,contentDetails',
                        id = ",".join(video_dict['video_ids'][50:])
                )
                response1 = request1.execute()
                video_info.extend(response1["items"])          

                return video_info

        except googleapiclient.errors.HttpError as e:
                status_code = e.resp.status
                error_message = e.content.decode("utf-8")
                print(f"HTTP status code: {status_code}")
                print(f"Error message: {error_message}")

In [ ]:
video_info = get_video_details(channel_video_dict)

In [ ]:
video_info

## Calculate time duration of videos

In [ ]:
def remove_shorts_videos(video_info):

    try:
        video_duration_seconds = []
        short_video_duration_threshold = 120
        non_short_videos = []

        for i in range(len(video_info)):
            if "S" in video_info[i]['contentDetails']['duration'] and "M" not in video_info[i]['contentDetails']['duration']:
                video_duration_seconds.append(int(video_info[i]['contentDetails']['duration'].split("T")[-1].split("S")[0]))

            if "H" in video_info[i]['contentDetails']['duration']:
                video_duration_seconds.append(int(video_info[i]['contentDetails']['duration'].split("T")[-1].split("H")[0]) * 3600)

            if "M" in video_info[i]['contentDetails']['duration'] and "H" not in video_info[i]['contentDetails']['duration']:
                video_duration_seconds.append(int(video_info[i]['contentDetails']['duration'].split("T")[-1].split("M")[0]) * 60)


        no_of_videos = len(non_short_videos)

        for i, duration in enumerate(video_duration_seconds):
            if no_of_videos >= 50:
                break

            if duration > short_video_duration_threshold:
                non_short_videos.append(channel_video_dict['video_ids'][i])
                no_of_videos += 1

        channel_video_info = dict(channel_id = channel_info['channel_id'], video_ids = non_short_videos)

        return channel_video_info

    except googleapiclient.errors.HttpError as e:
        status_code = e.resp.status
        error_message = e.content.decode("utf-8")
        print(f"HTTP status code: {status_code}")
        print(f"Error message: {error_message}")


In [ ]:
channel_video_list = remove_shorts_videos(video_info)

In [ ]:
def get_final_video_details(video_list):

    video_desc = {}
    final_video_info = []
    
    
    try:
        request = youtube.videos().list(
        part = 'snippet,statistics,contentDetails',
        id = ",".join(video_list['video_ids'])
        )
        response = request.execute()

        video_desc = dict(
        channel_ID = [response['items'][i]['snippet']['channelId'] for i in range(len(response['items']))],
        video_ID = [response['items'][i]['id'] for i in range(len(response['items']))],
        video_published_date = [dt.strptime(response['items'][i]['snippet']['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%d") for i in range(len(response['items']))],
        video_title = [response['items'][i]['snippet']['title'] for i in range(len(response['items']))],
        video_views = [response['items'][i]['statistics']['viewCount'] for i in range(len(response['items']))],
        video_likes = [response['items'][i]['statistics']['likeCount'] for i in range(len(response['items']))],
        video_comments = [response['items'][i]['statistics']['commentCount'] for i in range(len(response['items']))],
        video_thumbnail = [response['items'][i]['snippet']['thumbnails']['high']['url'] for i in range(len(response['items']))],
        video_links = ["https://www.youtube.com/watch?v="+response['items'][i]['id'] for i in range(len(response['items']))]
        )
        
        final_video_info.append(video_desc)

        return final_video_info

    except googleapiclient.errors.HttpError as e:
        status_code = e.resp.status
        error_message = e.content.decode("utf-8")
        print(f"HTTP status code: {status_code}")
        print(f"Error message: {error_message}")

In [ ]:
get_final_video_details(channel_video_list)

In [ ]:
channel_video_list

In [ ]:
request = youtube.videos().list(
        part = 'snippet,statistics,contentDetails',
        id = ",".join(channel_video_list['video_ids'])
)
response = request.execute()

In [ ]:
len(response['items'])

In [ ]:
len(response)

In [ ]:
for i in range(len(response['items'])):
    print(response['items'][i]['statistics']['viewCount'])

In [ ]:
channel_ID = [response['items'][i]['snippet']['channelId'] for i in range(len(response['items']))]
video_ID = [response['items'][i]['id'] for i in range(len(response['items']))]
video_published_date = [dt.strptime(response['items'][i]['snippet']['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%d") for i in range(len(response['items']))]
video_title = [response['items'][i]['snippet']['title'] for i in range(len(response['items']))]
video_views = [response['items'][i]['statistics']['viewCount'] for i in range(len(response['items']))]
video_likes = [response['items'][i]['statistics']['likeCount'] for i in range(len(response['items']))]
video_comments = [response['items'][i]['statistics']['commentCount'] for i in range(len(response['items']))]
video_thumbnail = [response['items'][i]['snippet']['thumbnails']['high']['url'] for i in range(len(response['items']))]
video_links = ["https://www.youtube.com/watch?v="+response['items'][i]['id'] for i in range(len(response['items']))]

In [ ]:
video_desc = {}
video_desc = dict(
channel_ID = [response['items'][i]['snippet']['channelId'] for i in range(len(response['items']))],
video_ID = [response['items'][i]['id'] for i in range(len(response['items']))],
video_published_date = [dt.strptime(response['items'][i]['snippet']['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%d") for i in range(len(response['items']))],
video_title = [response['items'][i]['snippet']['title'] for i in range(len(response['items']))],
video_views = [response['items'][i]['statistics']['viewCount'] for i in range(len(response['items']))],
video_likes = [response['items'][i]['statistics']['likeCount'] for i in range(len(response['items']))],
video_comments = [response['items'][i]['statistics']['commentCount'] for i in range(len(response['items']))],
video_thumbnail = [response['items'][i]['snippet']['thumbnails']['high']['url'] for i in range(len(response['items']))],
video_links = ["https://www.youtube.com/watch?v="+response['items'][i]['id'] for i in range(len(response['items']))]
)

In [ ]:
final_video_info = []
final_video_info.append(video_desc)

In [ ]:
for i in range(len(final_video_info)):
    print(final_video_info[i]['channel_ID'])

In [ ]:
len(video_data['video_ID'])

In [ ]:
count_video = 0
for i in video_data:
    print(len(['video_ID']))

In [ ]:
# import googleapiclient.errors

# try:
#     pass

# except googleapiclient.errors.HttpError as e:
#     status_code = e.resp.status
#     error_message = e.content.decode("utf-8")
#     print(f"HTTP status code: {status_code}")
#     print(f"Error message: {error_message}")